In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Data getting, cleaning, and exploring
import wrangle as w
import explore as ex

# Python without these is hard
import pandas as pd
import numpy as np
from pydataset import data

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
import sklearn.preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.impute import SimpleImputer



# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import graphviz
from graphviz import Graph

In [2]:
# w.get_zillow_data(cached=False)

In [3]:
df = w.get_zillow_data(cached=True)

In [4]:
df.shape

(20512, 62)

In [5]:
# there are a lot of nulls in this dataset, 
df.shape[0] - (df.shape[0] * .15)

17435.2

In [6]:
df = df.dropna(axis=1,thresh=17435)

In [7]:
target = 'taxvaluedollarcnt'

In [8]:
df.drop(columns=['calculatedbathnbr','calculatedfinishedsquarefeet','fullbathcnt','finishedsquarefeet12','lotsizesquarefeet'], inplace=True)

In [10]:
region_df = df[['regionidzip','regionidcity','regionidcounty','fips']]

In [ ]:
features = ['parcelid', 'bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'fips','latitude', 'longitude', 'regionidzip', 'yearbuilt', 'taxvaluedollarcnt', 'transactiondate']

In [ ]:
df = df[features]

In [ ]:
df.set_index('parcelid', inplace=True)

In [ ]:
df.columns

In [ ]:
# df.columns = ['bathrooms',
#              'bedrooms',
#              'square_feet',
#              'fips',
#              'latitude',
#              'longitude',
#              'zip_code',
#              'year_built',
#              'tax_value',
#              'transaction_date']

In [ ]:
# df = df.dropna()

In [ ]:
# df.info()

In [ ]:
# df.fips = df.fips.astype(int)

In [ ]:
# df.zip_code = df.zip_code.astype(int)

In [ ]:
# df.year_built = df.year_built.astype(int)

In [ ]:
# df['age_of_home'] = (2021 - df.year_built)

In [ ]:
# df.info()

In [ ]:
# df = ex.remove_outliers(df, 'square_feet', multiplier=1.5)

In [ ]:
# df = ex.remove_outliers(df, 'tax_value', multiplier=1.5)

In [ ]:
# df.info()

In [ ]:
df = w.wrangle_zillow()

In [ ]:
df.info()

In [ ]:
target = 'tax_value'

In [ ]:
w.select_kbest(df, target, 7)

In [ ]:
w.rfe(df,target,7)

# Wrangle above,
# Exploration Below

In [ ]:
train, validate, test = w.train_validate_test_split(df)

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
list(train.columns)

In [ ]:
target = 'tax_value'

In [ ]:
sns.pairplot(data = train, hue='tax_value')

In [ ]:
sns.relplot(data=train, y='latitude', x='longitude', hue='tax_value',)

In [ ]:
#fips 6111 = ventura co. 
#            coastal zips [93041, 93033, 93043, 93035, 93036, 93042 ]
#     6059 = Orange co.
#            coastal zips = [90740, 90743, 90742, 92649, 92648, 92646, 92663, 92662, 
#                            92625, 92657, 92651, 92629, 92624, 92672
#     6037 = Madera co.
#          


In [ ]:
# longitude and latitude to square_feet, target tax_value
sns.scatterplot(data=train, y='latitude',x= 'square_feet', hue='tax_value')

higher tax_value density towards south

In [ ]:
plt.figure(figsize=(14,10))


sns.jointplot(data = train, 
              x = 'tax_value', 
              y = 'latitude', 
              kind = 'kde', 
              cmap = 'Greens',
              fill = True)

plt.show()

sns.jointplot(data = train, 
              x = 'square_feet', 
              y = 'latitude', 
              kind = 'kde', 
              cmap = 'Blues',
              fill = True)

plt.show()

In [ ]:
plt.figure(figsize=(10,8))


sns.jointplot(data = train, 
              x = 'longitude', 
              y = 'tax_value', 
              kind = 'kde', 
              cmap = 'Greens',
              fill = True)

plt.show()

sns.jointplot(data = train, 
              x = 'longitude', 
              y = 'square_feet', 
              kind = 'kde', 
              cmap = 'Blues',
              fill = True)

plt.show()

perhaps higher tax_value to the east?

In [ ]:
sns.relplot(data=train, x='longitude',y='square_feet', hue='tax_value')

Scale the train data